In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
%matplotlib inline

In [2]:
test_dir = r'C:\Users\fabio\Desktop\Udemy\CNN for Computer Vision with Keras and Tensorflow\Creating CNN Model from scratch\test'
train_dir = r'C:\Users\fabio\Desktop\Udemy\CNN for Computer Vision with Keras and Tensorflow\Creating CNN Model from scratch\train'
validation_dir = r'C:\Users\fabio\Desktop\Udemy\CNN for Computer Vision with Keras and Tensorflow\Creating CNN Model from scratch\validation'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
#Generating batches of tensor image data

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 40,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size = (150, 150), batch_size = 20, class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size = (150, 150), batch_size = 20, class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
#HOW TO USE TRANSFER LEARNING HAPPENS HERE
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',
                 include_top = False, #This imports only the convolutional part
                 input_shape = (150,150,3))

58892288/58889256 [==============================] - 66s 1us/step


In [6]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [7]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [9]:
#conv_base.trainable = False we run this if we don't want to optimizer the weights of the vgg16

In [10]:
from tensorflow.keras.optimizers import *

model.compile(loss = 'binary_crossentropy',
             optimizer = RMSprop(lr = 2e-5),
             metrics = ['acc'])

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("CNN_transfer-{epoch:02d}.h5")

## O TREINAMENTO DEMORA DEMAIS, para ver os resultados consultar a aula

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch = 100,
                             epochs = 30,
                             validation_data = validation_generator,
                             validation_steps = 50,
                             callbacks = [checkpoint])

In [ ]:
pd.DataFrame(history.history).plot(figsize = (8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
history_salvar = pd.DataFrame(history.history)

hist_csv_file = "history.csv"
with open(hist_csv_file, mode = 'w') as f:
    history_salvar.to_csv(f)

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                 target_size = (150,150),
                                                 batch_size = 20,
                                                 class_mode = 'binary')

In [ ]:
model.evaluate_generator(test_generator, steps = 50) #50 = 1000 images / 20 (batch_size)

#Usando transfer learning, foi possível conseguir uma acc de quase 98%